## TEST ANNOY with parlement speech embeddings
see [here](https://github.com/spotify/annoy)

In [1]:
from annoy import AnnoyIndex
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
embeddings = pickle.load(open("kamer_debatten_recente_speeches_met_emb.pck", "rb"))

In [3]:
embeddings

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,...,502,503,504,505,506,507,508,509,510,511
0,2010-01-12,NaN,1,weekers,VVD,828.0,False,376,Voorzitter. In de week van de val van DSB heef...,NL-TweedeKamer,...,0.053838,-0.041742,-0.032260,-0.067037,-0.022552,-0.023394,-0.065473,-0.069716,0.012569,0.062962
1,2010-01-12,NaN,2,bos,PvdA,1234.0,False,260,Voorzitter. De laatste conclusie van de heer W...,NL-TweedeKamer,...,0.053762,-0.070165,-0.014723,-0.067824,-0.056783,-0.006215,-0.072602,-0.071131,-0.018375,0.080761
2,2010-01-12,NaN,3,weekers,VVD,828.0,False,204,Het gaat ons er niet om dat de minister zich b...,NL-TweedeKamer,...,0.069975,-0.066717,-0.014065,-0.068072,-0.066792,0.006887,-0.068825,-0.071827,0.018592,0.074508
3,2010-01-12,NaN,4,bos,PvdA,1234.0,False,226,Ik zal die zorgen overbrengen aan de heer Sche...,NL-TweedeKamer,...,0.082787,-0.077583,0.064040,-0.065484,-0.048932,-0.053128,-0.070319,-0.073950,0.039685,0.095769
4,2010-01-12,NaN,5,tony van dijck,PVV,298.0,False,107,Ook de fractie van de Partij voor de Vrijheid ...,NL-TweedeKamer,...,-0.019395,-0.025600,-0.051116,-0.067381,-0.071851,-0.021703,-0.082825,-0.032184,-0.049826,0.088014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468192,2019-07-04,NaN,963,staatssecretaris broekers-knol,NaN,NaN,False,290,Ik ga uit van de gegevens die wij hebben gekre...,NL-TweedeKamer,...,0.051056,-0.042262,-0.051334,-0.051260,-0.016387,-0.056653,-0.002518,-0.075303,-0.005929,0.087212
468193,2019-07-04,NaN,965,staatssecretaris broekers-knol,NaN,NaN,False,142,Dan de motie op stuk nr. 2519 van de heer Hidd...,NL-TweedeKamer,...,0.059314,-0.056630,-0.031098,-0.041214,-0.001191,0.002085,0.038942,-0.011882,0.044419,0.059911
468194,2019-07-04,NaN,967,de voorzitter,NaN,NaN,True,62,Over exact 60 minuten gaan wij stemmen over de...,NL-TweedeKamer,...,0.008260,-0.023593,0.078514,-0.011184,-0.030405,-0.023197,0.012507,-0.052127,0.023723,0.062293
468195,2019-07-04,NaN,968,mevrouw van toorenburg,CDA,1157.0,False,21,Toch nog even — misschien is het allemaal afge...,NL-TweedeKamer,...,-0.073596,0.022221,-0.085983,-0.041239,0.000995,-0.014434,-0.021121,-0.051808,0.041230,0.124054


## afstanden tussen rijen met scikit learn

In [4]:
def find_speeches(i:int, historic_speeches, K:int = 10) :
    
    testquery = historic_speeches.iloc[i:(i+1),12:524]
   
    #### look for the closest article embeddings, that we just created above
    #### the results object is an array with nart cosine similarities
    results = cosine_similarity(historic_speeches.iloc[:,12:524], testquery)

    indices = results.argsort(axis = 0).ravel()
    best_K = indices[-K:]

    ### take corresponding articles
    out = historic_speeches.iloc[best_K,:]
    out = (
        out
        .assign(similarity = results[best_K].flatten())
        .sort_values(by = "similarity", ascending = False)
    )
    return out[["datum", "speaker", "party", "text", "similarity"]]
    

In [5]:
%%time 
find_speeches(11, embeddings)

CPU times: user 1.7 s, sys: 618 ms, total: 2.32 s
Wall time: 1.89 s


,datum,speaker,party,text,similarity
11,2010-01-12,bos,PvdA,Mevrouw de voorzitter. Mevrouw Verdonk is een ...,1.000000
1,2010-01-12,bos,PvdA,Voorzitter. De laatste conclusie van de heer W...,0.764701
445203,2019-02-12,minister grapperhaus,NaN,Het is misschien goed om daar toch nog het vol...,0.702756
105887,2012-03-07,schippers,VVD,"Nee, ik heb de Kamer een onderzoek voorgesteld...",0.701216
117142,2012-05-15,aptroot,VVD,"Ja, ik rond af. Ik heb een laatste punt over d...",0.698580
300100,2015-12-16,minister van der steur,NaN,Ik stel vast dat de commissie-Oosting dat ook ...,0.695527
456505,2019-04-17,staatssecretaris keijzer,NaN,Ik zal mijn uiterste best voor u doen. In de m...,0.684091
99128,2012-01-31,opstelten,CDA,Op verzoek van de Kamer en in goed overleg met...,0.680848
302296,2016-01-20,minister van der steur,NaN,Ik vind dat de Kamer geïnformeerd moet worden....,0.679452
184499,2013-10-30,schippers,VVD,"Over de discussie, de debatten en de agenderin...",0.678469


## Now lets try annoy

In [6]:
f = 512
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

In [7]:
embeddings.iloc[1:2,12:524].values.flatten().shape

(512,)

In [8]:
N = embeddings.shape[0]
N

468197

In [9]:
%%time 
for i in range(N):
    v = embeddings.iloc[i:(i+1),12:524].values.flatten()
    t.add_item(i, v)

CPU times: user 3min 10s, sys: 1.39 s, total: 3min 12s
Wall time: 3min 12s


In [10]:
%%time
t.build(200)
t.save('test.ann')


CPU times: user 5min 29s, sys: 7.53 s, total: 5min 37s
Wall time: 5min 38s


True

In [11]:
%%time 
f = 512
u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file


CPU times: user 136 µs, sys: 188 µs, total: 324 µs
Wall time: 465 µs


True

In [19]:
 pp = u.get_nns_by_item(15, 25, include_distances=True)

In [23]:
%%time 

best_K = u.get_nns_by_item(15, 25, include_distances = True)
out = embeddings.iloc[best_K[0],:]
out["similarity"] = best_K[1]
out[["datum", "speaker", "party", "text", "similarity"]]
    

CPU times: user 5.85 ms, sys: 489 µs, total: 6.34 ms
Wall time: 5.53 ms


,datum,speaker,party,text,similarity
15,2010-01-12,bijleveld-schouten,CDA,Dit is bijna een herhaling van het debat dat g...,0.000345
13,2010-01-12,bijleveld-schouten,CDA,Voorzitter. Inderdaad is het zo dat de wijzigi...,0.746714
456094,2019-04-16,de heer hijink,SP,"Dank u wel, voorzitter. Ik sluit me wel aan bi...",0.778024
8139,2010-03-04,bijleveld-schouten,CDA,Ik ben het met u eens dat wat er in het rappor...,0.836932
374701,2017-11-02,minister rutte,NaN,"Wat wij hier willen doen, is het volgende. Ik ...",0.843060
98092,2012-01-25,van toorenburg,CDA,Voorzitter. Vorige sprekers hebben al uiteenge...,0.843815
319610,2016-05-19,de heer otwin van dijk,PvdA,De PvdA heeft altijd gezegd dat als de wet moe...,0.844131
313336,2016-03-30,mevrouw pia dijkstra,D66,Dat zal niet anders zijn dan het nu is. Elke a...,0.844673
181536,2013-10-09,leijten,SP,Samen bekijken hoe we van twee voorstellen een...,0.846629
282267,2015-09-23,rutte,VVD,Voorzitter. Ik zal herhalen wat ik al in eerst...,0.850845


In [14]:
best_K

[15,
 13,
 456094,
 8139,
 374701,
 98092,
 319610,
 313336,
 181536,
 282267,
 104973,
 104975,
 462834,
 175552,
 43775,
 249625,
 336375,
 165,
 131005,
 262970,
 455494,
 74067,
 48307,
 290752,
 44322]

In [17]:
%%time 
find_speeches(25, embeddings, K =25)

CPU times: user 1.64 s, sys: 667 ms, total: 2.31 s
Wall time: 1.88 s


,datum,speaker,party,text,similarity
25,2010-01-12,ter horst,PvdA,De voorbeeldkorpsen die staan in het rapportje...,1.000000
381835,2017-11-30,de heer van dam,CDA,"Voorzitter, dank u wel. Allereerst een woord v...",0.753841
175157,2013-09-03,marcouch,PvdA,Mevrouw Helder heeft een soft en een hard cita...,0.753738
369861,2017-09-13,de heer van dam,CDA,"Dank u wel, voorzitter. Het overgrote deel van...",0.721770
61166,2011-05-31,opstelten,CDA,Voorzitter. Dank aan de heer Çörüz voor de doo...,0.713361
8840,2010-03-11,laaper,PvdA,De minister is primair in de positie om antwoo...,0.711036
20729,2010-09-15,hirsch ballin,CDA,Mijnheer de voorzitter. Ik wil graag beginnen ...,0.706071
369863,2017-09-13,de heer van raak,SP,Laatst was ik op een vrijdag op een plein in A...,0.702382
369869,2017-09-13,minister blok,NaN,Voorzitter. De meeste sprekers besteden in hun...,0.698894
369866,2017-09-13,de heer gijs van dijk,PvdA,"Voorzitter, dank u wel. Door meerdere woordvoe...",0.695503
